In [6]:
import pandas as pd

# CSV 파일 읽기
df_target = pd.read_csv("표준코드/산지코드_직팜.csv", encoding='cp949')
df_source = pd.read_csv("기상/산지코드_직팜_new.csv", encoding='cp949')

# 위도/경도가 없을 경우를 대비해 컬럼이 없으면 생성
if '위도' not in df_target.columns:
    df_target['위도'] = None
if '경도' not in df_target.columns:
    df_target['경도'] = None

# 중복 제거 후 참조 딕셔너리 생성
coord_dict = df_source.drop_duplicates(subset='직팜산지이름') \
                      .set_index('직팜산지이름')[['위도', '경도']] \
                      .to_dict('index')

# 반복문으로 값 채우기 (이미 채워진 경우는 건너뜀)
for idx, row in df_target.iterrows():
    # 이미 위도/경도가 있으면 스킵
    if pd.notnull(row['위도']) and pd.notnull(row['경도']):
        continue

    name = row['직팜산지이름']
    if name in coord_dict:
        df_target.at[idx, '위도'] = coord_dict[name]['위도']
        df_target.at[idx, '경도'] = coord_dict[name]['경도']

# 결과 저장
df_target.to_csv("기상/산지코드_직팜_위경도_병합완료.csv", index=False, encoding='cp949')

In [2]:
import pandas as pd
from geopy.distance import geodesic

# 파일 경로
farm_path = "기상/산지코드_직팜_위경도_병합완료.csv"
station_path = "기상/기상청API_기상관측지점.csv"

# 파일 로딩
df_farms = pd.read_csv(farm_path, encoding='cp949')
df_stations = pd.read_csv(station_path, encoding='cp949')

# 컬럼명 공백 제거
df_stations.columns = df_stations.columns.str.strip()

# 관측지점 좌표 준비
station_coords = df_stations[['STN_ID', 'LON', 'LAT']].dropna().values

# 관측지점 STN_ID를 저장할 컬럼
df_farms['관측지점'] = None

# 각 산지에 대해 가장 가까운 관측소의 STN_ID 찾기
for idx, row in df_farms.iterrows():
    if pd.isna(row['위도']) or pd.isna(row['경도']):
        continue

    farm_coord = (row['위도'], row['경도'])

    # 거리 계산 후 가장 가까운 관측소 찾기
    closest_station = min(
        station_coords,
        key=lambda x: geodesic(farm_coord, (x[2], x[1])).meters
    )
    df_farms.at[idx, '관측지점'] = int(closest_station[0])  # STN_ID

# 저장
output_path = "기상/산지코드_직팜_관측지점_매핑완료.csv"
df_farms.to_csv(output_path, index=False, encoding='cp949')

In [5]:
import pandas as pd

# 파일 불러오기
df_weather = pd.read_csv("기상/기상청_일기요소.csv", encoding='cp949')

# 날짜 정제: 'YYYYMMDD' → datetime 형식으로 변환
df_weather['TM'] = pd.to_datetime(df_weather['TM'], format='%Y%m%d', errors='coerce')

# 그대로 저장 (컬럼명 유지)
df_weather.to_csv("기상/기상청_일기요소.csv", index=False, encoding='cp949')